### loading the libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing, ensemble
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier

#### create list for target and feature from  feature selection
##### *during the discussion it was adviced that fecho_dato with '2015-05-28' give good prediction so we are using this column inspite reported as false in rfe* 

In [2]:
feat_col = ["fecha_dato","ind_empleado","pais_residencia","sexo","age", "ind_nuevo", "antiguedad", "nomprov", 
            "cod_prov", "renta","indrel","segmento","ncodpers"]
target_col = ['ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1',
              'ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
                'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1',
              'ind_deco_fin_ult1','ind_deme_fin_ult1','ind_dela_fin_ult1',
              'ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1',
              'ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1',
              'ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1',
              'ind_nomina_ult1','ind_nom_pens_ult1','ind_recibo_ult1']

#### load the data set with features columns

In [3]:
train_1 = pd.read_csv("../input/santander-pr/train.csv", usecols=feat_col)
test = pd.read_csv("../input/santander-pr/test.csv", usecols=feat_col)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
import gc
gc.collect()

20

#### basic preprocessing and missing value imputation

In [5]:
train=train_1[(train_1['fecha_dato'] == '2015-05-28')]
gc.collect()

20

In [6]:
train

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,ind_nuevo,antiguedad,indrel,cod_prov,nomprov,renta,segmento
2512427,2015-05-28,1061260,N,ES,H,24,0.0,34,1.0,28.0,MADRID,396426.42,03 - UNIVERSITARIO
2512428,2015-05-28,1061283,N,ES,H,22,0.0,34,1.0,28.0,MADRID,155917.08,03 - UNIVERSITARIO
2512429,2015-05-28,1061284,N,ES,H,24,0.0,34,1.0,36.0,PONTEVEDRA,NaN,03 - UNIVERSITARIO
2512430,2015-05-28,1061336,N,ES,H,23,0.0,34,1.0,27.0,LUGO,66804.60,03 - UNIVERSITARIO
2512431,2015-05-28,1061286,N,ES,V,29,0.0,34,1.0,41.0,SEVILLA,106102.68,03 - UNIVERSITARIO
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3144379,2015-05-28,547712,N,ES,V,48,0.0,119,1.0,20.0,GIPUZKOA,NaN,02 - PARTICULARES
3144380,2015-05-28,547691,N,ES,H,36,0.0,119,1.0,11.0,CADIZ,90132.09,02 - PARTICULARES
3144381,2015-05-28,1399490,N,ES,H,2,1.0,2,1.0,47.0,VALLADOLID,130692.45,02 - PARTICULARES
3144382,2015-05-28,15889,F,ES,V,56,0.0,245,1.0,28.0,MADRID,326124.90,01 - TOP


In [7]:
del train_1

In [8]:
gc.collect()

40

#### Dimension of the dataset

In [9]:
print(train.shape)
print(test.shape)

(631957, 13)
(931453, 13)


In [10]:
train.isnull().sum()

fecha_dato              0
ncodpers                0
ind_empleado         3597
pais_residencia      3597
sexo                 3600
age                     0
ind_nuevo            3597
antiguedad              0
indrel               3597
cod_prov             7275
nomprov              7275
renta              109919
segmento             3619
dtype: int64

#### As we are using XgBOOST so according to the alogoithm we have to fill the missing values with zero

In [11]:
train.fillna(0,inplace=True)

In [12]:
train["renta"]   = pd.to_numeric(train["renta"], errors="coerce")
test["renta"]   = pd.to_numeric(test["renta"], errors="coerce")

In [13]:
test.fillna(0,inplace=True)

In [14]:
test

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,ind_nuevo,antiguedad,indrel,cod_prov,nomprov,renta,segmento
0,2016-05-28,657640,N,ES,H,68,0.0,114,1.0,28.0,MADRID,147347.97,02 - PARTICULARES
1,2016-05-28,657788,N,ES,H,42,0.0,114,1.0,28.0,MADRID,132559.35,02 - PARTICULARES
2,2016-05-28,657795,N,ES,V,44,0.0,114,1.0,26.0,"RIOJA, LA",81399.57,02 - PARTICULARES
3,2016-05-28,657790,N,ES,V,42,0.0,114,1.0,48.0,BIZKAIA,0.00,02 - PARTICULARES
4,2016-05-28,657794,N,ES,H,49,0.0,114,1.0,8.0,BARCELONA,102189.00,01 - TOP
...,...,...,...,...,...,...,...,...,...,...,...,...,...
931448,2016-05-28,1166765,N,ES,V,22,0.0,33,1.0,50.0,ZARAGOZA,43912.17,03 - UNIVERSITARIO
931449,2016-05-28,1166764,N,ES,V,23,0.0,33,1.0,26.0,"RIOJA, LA",23334.99,03 - UNIVERSITARIO
931450,2016-05-28,1166763,N,ES,H,47,0.0,33,1.0,50.0,ZARAGOZA,0.00,02 - PARTICULARES
931451,2016-05-28,1166789,N,ES,H,22,0.0,33,1.0,50.0,ZARAGOZA,199592.82,03 - UNIVERSITARIO


#### change the type and replacing the out liers for better data processing

In [15]:
train["age"]   = pd.to_numeric(train["age"], errors="coerce")
test["age"]   = pd.to_numeric(test["age"], errors="coerce")
train['antiguedad'].replace('     NA',np.nan,inplace=True)
train['antiguedad'].replace('-999999',np.nan,inplace=True)
train['antiguedad'].replace(-999999,np.nan,inplace=True)
train.antiguedad = pd.to_numeric(train.antiguedad, errors='coerce')
train.antiguedad = train.antiguedad.astype('float16')

test['antiguedad'].replace('     NA',np.nan,inplace=True)
test['antiguedad'].replace('-999999',np.nan,inplace=True)
test['antiguedad'].replace(-999999,np.nan,inplace=True)
test.antiguedad = pd.to_numeric(test.antiguedad, errors='coerce')
test.antiguedad = test.antiguedad.astype('float16')
train.cod_prov = pd.to_numeric(train.cod_prov, errors='coerce')
train.cod_prov = train.cod_prov.astype('float16')

test.cod_prov = pd.to_numeric(test.cod_prov, errors='coerce')
test.cod_prov = test.cod_prov.astype('float16')

In [18]:
print(train.shape)

(631957, 13)


In [19]:
train.isnull().sum()

fecha_dato            0
ncodpers              0
ind_empleado          0
pais_residencia       0
sexo                  0
age                3597
ind_nuevo             0
antiguedad         3599
indrel                0
cod_prov              0
nomprov               0
renta                 0
segmento              0
dtype: int64

In [20]:
train.fillna(0,inplace=True)
test.fillna(0,inplace=True)

In [21]:
train.isnull().sum()

fecha_dato         0
ncodpers           0
ind_empleado       0
pais_residencia    0
sexo               0
age                0
ind_nuevo          0
antiguedad         0
indrel             0
cod_prov           0
nomprov            0
renta              0
segmento           0
dtype: int64

In [22]:
train.segmento.value_counts()

02 - PARTICULARES     368666
03 - UNIVERSITARIO    227714
01 - TOP               31958
0                       3619
Name: segmento, dtype: int64

In [23]:
train.isnull().sum()

fecha_dato         0
ncodpers           0
ind_empleado       0
pais_residencia    0
sexo               0
age                0
ind_nuevo          0
antiguedad         0
indrel             0
cod_prov           0
nomprov            0
renta              0
segmento           0
dtype: int64

In [24]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 631957 entries, 2512427 to 3144383
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   fecha_dato       631957 non-null  object 
 1   ncodpers         631957 non-null  int64  
 2   ind_empleado     631957 non-null  object 
 3   pais_residencia  631957 non-null  object 
 4   sexo             631957 non-null  object 
 5   age              631957 non-null  float64
 6   ind_nuevo        631957 non-null  float64
 7   antiguedad       631957 non-null  float16
 8   indrel           631957 non-null  float64
 9   cod_prov         631957 non-null  float16
 10  nomprov          631957 non-null  object 
 11  renta            631957 non-null  float64
 12  segmento         631957 non-null  object 
dtypes: float16(2), float64(4), int64(1), object(6)
memory usage: 60.3+ MB


#### create a list to label encode the object values

In [25]:
lisobj = []
for ind, col in enumerate(train.columns):
    if train[col].dtype == "object":
        lisobj.append(col)

In [26]:
lisobj

['fecha_dato',
 'ind_empleado',
 'pais_residencia',
 'sexo',
 'nomprov',
 'segmento']

#### label encoding 

In [27]:
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import make_column_transformer

In [29]:
le = LabelEncoder()

In [30]:
for i in lisobj:
    train[i]=train[i].astype('str')
    train[i]=le.fit_transform(train[i])

In [31]:
for i in lisobj:
    test[i]=test[i].astype('str')
    test[i]=le.fit_transform(test[i])

In [32]:
import gc
gc.collect()

1438

In [33]:
train_Y = pd.read_csv("../input/santander-pr/train.csv", usecols=['ncodpers','fecha_dato']+target_col)
train_Y.shape

(12715856, 26)

In [34]:
train_Y=train_Y[train_Y['fecha_dato'] == '2015-05-28']
train_Y.drop(columns='fecha_dato',inplace=True)
gc.collect()

20

In [35]:
train_Y.fillna(0,inplace=True)

In [36]:
tar=[  'ind_ahor_fin_ult1', 'ind_aval_fin_ult1',
       'ind_cco_fin_ult1', 'ind_cder_fin_ult1', 'ind_cno_fin_ult1',
       'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1',
       'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_deme_fin_ult1',
       'ind_dela_fin_ult1', 'ind_ecue_fin_ult1', 'ind_fond_fin_ult1',
       'ind_hip_fin_ult1', 'ind_plan_fin_ult1', 'ind_pres_fin_ult1',
       'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1',
       'ind_viv_fin_ult1', 'ind_nomina_ult1', 'ind_nom_pens_ult1',
       'ind_recibo_ult1']

## model building and prediction

#### model build with XGBoost
#### *for loops’ to train the model against each target feature using train dataset and predict the same using test dataset*

In [37]:
prob=[]
for i in tar:
    clf = xgb.XGBClassifier(max_depth=12, learning_rate = 0.05, 
                 subsample = 0.7, colsample_bytree = 0.7, n_estimators=70,
                 colsample_bylevel= 0.7, min_child_weight = 5,
               nthread=4)
    clf.fit(train,train_Y[i])
    y=clf.predict(test)
    prob.append(clf.predict_proba(test)[:,1])
    clf=0
    undersampled_data = 0
    result = 0
    gc.collect()

In [38]:
len(prob)

24

In [39]:
gc.collect()

40

In [40]:
preds = np.array(prob)[:,:].T

In [41]:
train_Y = pd.read_csv("../input/santander-pr/train.csv", usecols=['ncodpers','fecha_dato']+target_col)
train_Y=train_Y[train_Y['fecha_dato'] == '2016-04-28']
train_Y.drop(columns='fecha_dato',inplace=True)
gc.collect()

0

### *separate the target features into two groups based on their last month data being ‘0’ or ‘1’*

In [42]:
last_instance_df =  train_Y
cust_dict = {}
cust_undict= {}
target_cols = np.array(target_col)
for ind, row in last_instance_df.iterrows():
    cust = row['ncodpers']
    used_products = set(target_cols[np.array(row[1:])==1])
    unused_products = set(target_cols[np.array(row[1:])==0])
    cust_dict[cust] = used_products
    cust_undict[cust] = unused_products
del (last_instance_df)

#### *prediction*

In [43]:
final_1 = np.argsort(preds, axis=1)
final_2 = np.fliplr(final_1)
test_id = np.array(pd.read_csv("../input/santander-pr/test.csv", usecols=['ncodpers'])['ncodpers'])
final_preds = []
for ind, pred in enumerate(final_2):
    temp_dict={}
    #print("--------------------------------------")
    cust = test_id[ind]
    top_products = target_cols[pred]
    drop_products = target_cols[pred[::-1]]
    used_products = cust_dict.get(cust,[])
    unused_products = cust_undict.get(cust,[])
    new_top_products = []
    new_drop_products = []
    product_col = []
    pred_col = []
    use_unuse = []
    diffrence = []
    rank = []
    # customers after index 928274 are new and 
    #thus do not have previous month data to compare the probablities
    # to compare to the target variables of the previous month
    # thus we process their probabilities differently
    if ind >= 928274:                   
        counter = 0
        for product in drop_products: 
            if product not in unused_products:
                diffrence.append(1 - preds[ind][pred[::-1][counter]])
                new_top_products.append(product)
            counter += 1
        df = pd.DataFrame()
        df['col_1'] = new_top_products
        df['col_2'] = diffrence
        new_top_products = df.sort_values('col_2', ascending = False).col_1.head().to_list()
    #
    else:
        #we separate the target features into two groups 
        #based on their last month data being ‘0’ or ‘1’ 
        counter = 0
        for product in drop_products:
            if product not in unused_products:
                #for all the target features segmented into the ‘1’ category the resultant 
                #probability (of being ‘1’) of that feature given by 
                #model is subtracted to 1 (giving probability of being ‘0’ i.e. dropping product). 
                diffrence.append(1 - preds[ind][pred[::-1][counter]])
                new_top_products.append(product)
            counter += 1
        counter = 0
        for product in top_products: 
            if product not in used_products:
                diffrence.append(preds[ind][pred[counter]])
                new_top_products.append(product)
            counter += 1
        #all the probabilities are compared and the top 5 features with highest probability
        #(of changing) are given as output prediction for that customer. 

        df = pd.DataFrame()
        df['col_1'] = new_top_products  
        df['col_2'] = diffrence
        new_top_products = df.sort_values('col_2', ascending = False).col_1.head().to_list()    
    final_preds.append(" ".join(new_top_products))
out_df = pd.DataFrame({'ncodpers':test_id, 'changed':final_preds})
out_df.to_csv('sub_prev_m4.csv', index=False)